In [15]:
import pandas as pd

In [16]:
dev_mentions = pd.read_csv('dataset/dev_mentions_gold.tsv', sep='\t')

In [17]:
train_dictionary_unambiguous = pd.read_csv('dataset/train_dictionary_unambiguous.tsv', sep='\t')

In [18]:
train_dictionary_unambiguous.head()

,code,term,term_original,is_unambiguous
0,J00,κοινό κρυολόγημα,Κοινό κρυολόγημα,True
1,J00,οξεία ρινοφαρυγγίτιδα,Οξεία ρινοφαρυγγίτιδα,True
2,J00,λοιμώδης ρινίτιδα,Λοιμώδης ρινίτιδα,True
3,J00,οξεία ρινίτιδα,Οξεία ρινίτιδα,True
4,J00,ρινίτιδα (λοιμώδης),Ρινίτιδα (λοιμώδης),True


In [19]:
el_gold = dev_mentions
el_gold['mention'] = el_gold['text'].apply(lambda x: x.lower())

In [20]:
umls_term_texts = train_dictionary_unambiguous['term'].tolist()

filter_direct_match = ~(el_gold['mention'].isin(umls_term_texts))

el_gold = el_gold[filter_direct_match]
len(el_gold)

693

In [22]:
def predict(mention):
    if mention in umls_term_texts: 
        return train_dictionary_unambiguous[train_dictionary_unambiguous['term']==mention].iloc[0]['code']
    return ""

In [23]:
dev_predictions = dev_mentions
dev_predictions['ann_id']='ICD'
dev_predictions['pred_code'] = dev_predictions['mention'].apply(lambda x: predict(x.lower()))
dev_predictions.head()

,filename,ann_id,label,start_span,end_span,text,code,mention,pred_code
0,3,ICD,ICD,220,226,NSTEMI,I21,nstemi,
1,3,ICD,ICD,530,544,στεφανιογραφία,Y84,στεφανιογραφία,
2,3,ICD,ICD,549,563,αγγειοπλαστική,Z95,αγγειοπλαστική,Z95
3,3,ICD,ICD,1003,1017,Στεφανιογραφία,Y84,στεφανιογραφία,
4,3,ICD,ICD,1020,1034,Αγγειοπλαστική,Z95,αγγειοπλαστική,Z95


In [26]:
dev_predictions.to_csv('dataset/dev_mentions_dictionary_match.tsv', index=False, sep='\t')